In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import wikipedia
import xml.etree.ElementTree as ET
import re
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from sklearn.model_selection import cross_val_score, KFold
import xgboost as xgb
from sklearn.metrics import r2_score

%matplotlib inline

In [2]:
df = pd.read_csv('sysarmy2018-1.csv')
df = df[df['Bruto o neto'] == 'Bruto']
sal_col = 'Salario mensual (en tu moneda local)'
df = df[(df[sal_col].astype(float) > 100) & (df[sal_col].astype(float) <= 1_000_000)]
df = df[df['Me identifico'] != 'Otros']
df.head()

,Me identifico,Tengo,Argentina,Anos de experiencia,Anos en el puesto actual,Gente a cargo,Cuanta,Nivel de estudios alcanzado,Estado,Carrera,...,Tecnologias que utilizas.3,Tecnologias que utilizas.4,Que tan conforme estas con tu sueldo,Como crees que esta tu sueldo con respecto a julio 2017,Recibis algun tipo de bono,A que esta atado el bono,Tuviste ajustes por inflacion en 2017,De que % fue el ajuste,Cambiaste de empresa en los ultimos 6 meses,Cual fue el principal motivo de cambio
0,Hombre,24 - 26,Cordoba,10+,Menos de un ano,Si,1.0,Universitario,Completado,Diseno Interactivo,...,NaN,NaN,1.0,1.0,No,No recibo bono,No,0.0,No,No cambie de empresa
3,Hombre,21 - 23,Provincia de Buenos Aires,3,3,No,0.0,Universitario,En curso,Lic. en Cs de la Comp,...,NaN,NaN,2.0,2.0,No,No recibo bono,Uno,5.0,No,No cambie de empresa
4,Hombre,21 - 23,GBA,Menos de un ano,Menos de un ano,No,0.0,Universitario,En curso,Ing Sistemas,...,NaN,NaN,3.0,2.0,No,No recibo bono,Uno,8.0,No,No cambie de empresa
7,Hombre,31 - 33,Ciudad Autonoma de Buenos Aires,5,2,No,0.0,Posgrado,En curso,NaN,...,NaN,NaN,2.0,2.0,Un sueldo,Performance individual,Dos,20.0,No,No cambie de empresa
9,Hombre,27 - 30,Provincia de Buenos Aires,2,1,No,0.0,Universitario,En curso,Computacion,...,NaN,NaN,1.0,1.0,No,No recibo bono,No,0.0,No,No cambie de empresa


In [3]:
h, m = (
df[df['Me identifico'] == 'Hombre'][sal_col].median(),
df[df['Me identifico'] == 'Mujer'][sal_col].median(),
)
(h-m)/h

0.2558139534883721

In [4]:
df.columns

Index(['Me identifico', 'Tengo', 'Argentina', 'Anos de experiencia',
       'Anos en el puesto actual', 'Gente a cargo', 'Cuanta',
       'Nivel de estudios alcanzado', 'Estado', 'Carrera',
       'Realizaste cursos de especializacion', 'Trabajo de',
       'Tipo de contrato', 'Salario mensual (en tu moneda local)', 'USD',
       'Bruto o neto', 'Max Cantidad de empleados', 'Tecnologias que utilizas',
       'Tecnologias que utilizas.1', 'Tecnologias que utilizas.2',
       'Automation o funcional', 'Tecnologias que utilizas.3',
       'Tecnologias que utilizas.4', 'Que tan conforme estas con tu sueldo',
       'Como crees que esta tu sueldo con respecto a julio 2017',
       'Recibis algun tipo de bono', 'A que esta atado el bono',
       'Tuviste ajustes por inflacion en 2017', 'De que % fue el ajuste',
       'Cambiaste de empresa en los ultimos 6 meses',
       'Cual fue el principal motivo de cambio'],
      dtype='object')

In [5]:
best = {'colsample_bytree': 0.7000000000000001, 'gamma': 0.8500000000000001, 'learning_rate': 0.025, 'max_depth': 16, 'min_child_weight': 15.0, 'n_estimators': 175, 'subsample': 0.8099576733552297}
regions_map = {
    'Ciudad Autonoma de Buenos Aires': 'AMBA',
    'GBA': 'AMBA',
    'Catamarca': 'NOA',
    'Chaco': 'NEA',
    'Chubut': 'Patagonia',
    'Corrientes': 'NEA',
    'Entre Rios': 'NEA',
    'Formosa': 'NEA',
    'Jujuy': 'NOA',
    'La Pampa': 'Pampa',
    'La Rioja': 'NOA',
    'Mendoza': 'Cuyo',
    'Misiones': 'NEA',
    'Neuquen': 'Patagonia',
    'Rio Negro': 'Patagonia',
    'Salta': 'NOA',
    'San Juan': 'Cuyo',
    'San Luis': 'Cuyo',
    'Santa Cruz': 'Patagonia',
    'Santa Fe': 'Pampa',
    'Santiago del Estero': 'NOA',
    'Tucuman': 'NOA',
    'Cordoba': 'Pampa',
    'Provincia de Buenos Aires': 'Pampa',
    'Tierra del Fuego': 'Patagonia',
}
class Model:
    def __init__(self, **params):
        self.regressor_ = xgb.XGBRegressor(**params)

    def get_params(self, deep=True):
        return self.regressor_.get_params(deep=deep)

    def set_params(self, **params):
        return self.regressor_.set_params(**params)
    
    def clean_words(self, field, value):
        value = value.replace('Microsoft Azure (Tables, CosmosDB, SQL, etc)', 'Microsoft Azure(TablesCosmosDBSQLetc)')
        value = value.replace('Snacks, golosinas, bebidas', 'snacks')
        value = value.replace('Descuentos varios (Clarín 365, Club La Nación, etc)', 'descuentos varios')
        value = value.replace('Sí, de forma particular', 'de forma particular')
        value = value.replace('Sí, los pagó un empleador', 'los pagó un empleador')
        value = value.replace('Sí, activa', 'activa')
        value = value.replace('Sí, pasiva', 'pasiva')
        return [self.clean_word(field, v) for v in value.split(',') if self.clean_word(field, v)]

    def clean_word(self, field, word):
        val = str(word).lower().strip().replace(".", "")
        if val in ('ninguno', 'ninguna', 'no', '0', 'etc)', 'nan'):
            return ''
        if field == 'Lenguajes de programación' and val == 'Microsoft Azure(TablesCosmosDBSQLetc)':
            return 'Microsoft Azure (Tables, CosmosDB, SQL, etc)'
        if field == '¿A qué eventos de tecnología asististe en el último año?' and val in ('pycon', 'pyconar'):
            return 'pyconar'
        if field == '¿A qué eventos de tecnología asististe en el último año?' and val in ('nodeconf', 'nodeconfar'):
            return 'nodeconfar'
        if field == '¿A qué eventos de tecnología asististe en el último año?' and val in ('meetup', 'meetups'):
            return 'meetups'
        if field == '¿A qué eventos de tecnología asististe en el último año?':
            return val.replace(' ', '')
        if field == 'Beneficios extra' and val == 'snacks':
            return 'snacks, golosinas, bebidas'
        if field == 'Beneficios extra' and val == 'descuentos varios':
            return 'descuentos varios (clarín 365, club la nación, etc)'
        return val

    def row_to_words(self, row):
        return [
            f'{key}={row.fillna("")[key]}'
            for key
            in (
                'Me identifico',
                'Trabajo de',
                'Tipo de contrato',
                'Nivel de estudios alcanzado',
                'Estado',
                'Carrera',
                'Max Cantidad de empleados',
            )
        ] + [
            f'{k}={v}' for k in (
                'Tecnologias que utilizas',
                'Tecnologias que utilizas.1',
                'Tecnologias que utilizas.2',
                'Tecnologias que utilizas.3',
                'Tecnologias que utilizas.4',
            ) for v in self.clean_words(k, row.fillna('')[k])
        ] + [
            f'region={regions_map[row["Argentina"]]}'
        ]

    def encode_row(self, row):
        ws = self.row_to_words(row)
        row = pd.Series([w in ws for w in self.valid_words_] + [
            int(row['Cuanta']),
            int(row['Anos de experiencia'].replace('+', '').replace('Menos', '0').split(' ')[0]),
            int(row['Tengo'].replace('+', '').replace('Menos', '0').split(' ')[0]),
        ])
        return row

    def fit(self, X, y, **params):
        counts = {}
        for i in range(X.shape[0]):
            for word in self.row_to_words(X.iloc[i]):
                counts[word] = counts.get(word, 0) + 1
        self.valid_words_ = [word for word, c in counts.items() if c > 0.01*X.shape[0]]
        self.regressor_.fit(X.apply(self.encode_row, axis=1).astype(float), y, **params)
        return self
   
    def predict(self, X):
        return self.regressor_.predict(X.apply(self.encode_row, axis=1).astype(float))
    
    def score(self, X, y):
        return r2_score(y, self.predict(X))

In [6]:
kf = KFold(n_splits=5, shuffle=True, random_state=99)
kf_models = []
for train_index, test_index in kf.split(df):
    model = Model(**best).fit(df.iloc[train_index], df.iloc[train_index][sal_col].astype(float))
    df.loc[df.index[test_index], 'e(salary)'] = model.predict(df.iloc[test_index])
    df['Me identifico'] = df['Me identifico'].apply(lambda g: {'Hombre': 'Mujer', 'Mujer': 'Hombre'}[g])
    df.loc[df.index[test_index], 'e_gr(salary)'] = model.predict(df.iloc[test_index])
    df['Me identifico'] = df['Me identifico'].apply(lambda g: {'Hombre': 'Mujer', 'Mujer': 'Hombre'}[g])
    kf_models.append(model)
df['e_h(salary)'] = df.apply(lambda row: row['e(salary)'] if row['Me identifico'] == 'Hombre' else row['e_gr(salary)'], axis=1)
df['e_m(salary)'] = df.apply(lambda row: row['e(salary)'] if row['Me identifico'] == 'Mujer' else row['e_gr(salary)'], axis=1)
df['e_g_diff(salary)'] = (df['e_h(salary)'] - df['e_m(salary)']) / df['e_h(salary)']
r2_score(df[sal_col], df['e(salary)'])

0.5143317828072063

In [7]:
df['e_g_diff(salary)'].median()

0.1178968318575872